In [114]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import os

In [115]:
df = pd.read_csv("../../ShoppersSmart/merged_dataset.csv")
df.head()

,transaction_id,customer_id,product_id,product_name,purchase_date,location,price,email,phone_number
0,1,42,84,Ceiling Fan - Premium,2025-06-19,Lucknow,1397.48,aadhya.nair@example.com,9025501427
1,2,55,83,Stapler Pack,2025-05-24,Mumbai,997.22,ishaan.nair@example.com,9768045202
2,3,87,214,Scissors (Combo),2025-06-12,Patna,1171.84,myra.chatterjee@example.com,9779426304
3,4,15,169,Memory Card XL,2025-06-25,Mumbai,1922.17,riya.nair@example.com,9101612239
4,5,78,179,Broom (Combo),2025-05-23,Delhi,683.59,myra.nair@example.com,9319718430


In [116]:
print(df.columns)

Index(['transaction_id', 'customer_id', 'product_id', 'product_name',
       'purchase_date', 'location', 'price', 'email', 'phone_number'],
      dtype='object')


In [117]:
recommendation = (
    df.groupby(['location','product_id' ])
    .size()
    .reset_index(name='count')
    .sort_values(['count'], ascending=[False])
)

In [118]:
recommendation = recommendation.merge(df[['product_id','product_name']],on='product_id')

In [119]:
recommendation_dict = {}

for loc in recommendation['location'].unique():
    top_products = (
        recommendation[recommendation['location'] == loc]
        .sort_values(by='count', ascending=False)
        .head(5)['product_name']
        .tolist()
    )
    recommendation_dict[loc.lower()] = top_products 

In [120]:
recommend_by_location('Pune',top_n=5)

,product_id,product_name
197,192,Tawa
221,153,Hair Oil - New
231,61,Lotion Set
230,61,Lotion Set
229,66,Mug


In [126]:
with open("location_recommendation.pkl", "wb") as f:
    pickle.dump(recommendation_dict, f)

In [127]:
recommendation[recommendation['location']=='Pune']

,location,product_id,count,product_name
197,Pune,192,1,Tawa
198,Pune,192,1,Tawa
199,Pune,192,1,Tawa
208,Pune,220,1,Frying Pan - New
209,Pune,220,1,Frying Pan - New
210,Pune,220,1,Frying Pan - New
211,Pune,195,1,File Folder Set
212,Pune,175,1,Refrigerator XL
213,Pune,175,1,Refrigerator XL
214,Pune,175,1,Refrigerator XL


In [128]:
print(recommendation['location'].unique())

['Indore' 'Ahmedabad' 'Bangalore' 'Hyderabad' 'Chennai' 'Delhi' 'Kolkata'
 'Lucknow' 'Jaipur' 'Patna' 'Nagpur' 'Pune' 'Surat' 'Mumbai' 'Bhopal']
